# COMPONENT 1 - WEB SCRAPING MODULE

### IMPORTING THE LIBRARIES

In [1]:
from bs4 import BeautifulSoup
import requests
import json
import pandas as pd

In [2]:
num_list = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12", "13", "14"]

### EXTRACTING NEWS DATA FOR TWO WEEKS (Feb 1, 2021 to Feb 14, 2021)

In [3]:
#ITERATING THROUGH THE 14 URLs TO EXTRACT THE NEWS DATA
for i in range(len(num_list)):
  page = requests.get("https://www.thehindu.com/archive/web/2021/02/{0}/".format(num_list[i]))
  soup = BeautifulSoup(page.content, "html.parser")
  li_list = {"text" : []}
  for ul in soup.find_all('ul', class_='archive-list'):
    for li in ul.find_all('li'):
        li_list["text"].append(li.text)
  with open('thehindu_feb_02_file_{0}.json'.format(num_list[i]), 'w') as jsonfile:
    json.dump(li_list, jsonfile)

### CREATING A DATAFRAME TO STORE THE FILE NAMES AND THE CORRESPONDING URLs

In [4]:
file_names = []
for i in range(len(num_list)):
  file_names.append('thehindu_feb_02_file_{0}.json'.format(num_list[i]))

url_list = []
for i in range(len(num_list)):
  url_list.append("https://www.thehindu.com/archive/web/2021/02/{0}/".format(num_list[i]))

tab_head = ["File_Scraped", "Corresponding URL"]
df_data = pd.DataFrame(zip(file_names, url_list), columns = tab_head)
print(df_data)

                    File_Scraped                                 Corresponding URL
0   thehindu_feb_02_file_01.json  https://www.thehindu.com/archive/web/2021/02/01/
1   thehindu_feb_02_file_02.json  https://www.thehindu.com/archive/web/2021/02/02/
2   thehindu_feb_02_file_03.json  https://www.thehindu.com/archive/web/2021/02/03/
3   thehindu_feb_02_file_04.json  https://www.thehindu.com/archive/web/2021/02/04/
4   thehindu_feb_02_file_05.json  https://www.thehindu.com/archive/web/2021/02/05/
5   thehindu_feb_02_file_06.json  https://www.thehindu.com/archive/web/2021/02/06/
6   thehindu_feb_02_file_07.json  https://www.thehindu.com/archive/web/2021/02/07/
7   thehindu_feb_02_file_08.json  https://www.thehindu.com/archive/web/2021/02/08/
8   thehindu_feb_02_file_09.json  https://www.thehindu.com/archive/web/2021/02/09/
9   thehindu_feb_02_file_10.json  https://www.thehindu.com/archive/web/2021/02/10/
10  thehindu_feb_02_file_11.json  https://www.thehindu.com/archive/web/2021/02/11/
11  

### STORING THE DATAFRAME IN .CSV FILE

In [5]:
df_data.to_csv('URLs_to_file_mapping.csv', sep ='|')
new_df = pd.read_csv('URLs_to_file_mapping.csv')
new_df

,|File_Scraped|Corresponding URL
0,0|thehindu_feb_02_file_01.json|https://www.the...
1,1|thehindu_feb_02_file_02.json|https://www.the...
2,2|thehindu_feb_02_file_03.json|https://www.the...
3,3|thehindu_feb_02_file_04.json|https://www.the...
4,4|thehindu_feb_02_file_05.json|https://www.the...
5,5|thehindu_feb_02_file_06.json|https://www.the...
6,6|thehindu_feb_02_file_07.json|https://www.the...
7,7|thehindu_feb_02_file_08.json|https://www.the...
8,8|thehindu_feb_02_file_09.json|https://www.the...
9,9|thehindu_feb_02_file_10.json|https://www.the...


### VIEWING THE RAW DATA

In [6]:
raw_text = []
for json_file in file_names:
  f = open(json_file,)
  data = json.load(f)
  print(data)
  raw_text.append(data) 

{'text': ['\nVan Foreest triumphs; Hari seventh\n', '\nAdvantage India \n', '\nMurray River Open | Nagal loses season opener\n', '\nChip sales rose 6.5% globally in 2020 after year-end rush\n', '\nElon Musk says bitcoin "on the verge" of being more widely accepted\n', '\nStudying e-cigarette using pattern with sensors and AI\n', "\nEU to rule on Microsoft's $7.5 bln ZeniMax deal by March 5\n", '\nMicrosoft Teams gets Approvals, Tasks publishing features\n', '\nApple iMessage gets new security protection in iOS 14 update\n', '\nHere’s how you can transfer WhatsApp chats to Telegram\n', '\nGoogle allows gambling apps in 15 countries; India not included\n', "\nWhat GameStop's surge says about online mobs\n", '\nRussian hack brings changes, uncertainty to US court system\n', '\nTaiwan to discuss supply chains with U.S. next week amid auto shortage\n', '\nTop news of the day: In Union Budget 2021, Finance Minister announces ₹64,180 crore healthcare scheme; poll-bound States to get new roads